# Data Cleaning and Pre-Processing

### Import libraries needed

In [1]:
import pandas as pd

In [2]:
state_cases_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')

state_cases_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [ ]:
# calculate case fatality rate
state_cases_df['cfr'] = state_cases_df.cases/state_cases_df.deaths